In [1]:
library("data.table")

# Drop close relatives

In [2]:
close_rel <- fread("/mnt/project/genotypes/ids_to_drop_close_relatives.txt")[, .(eid = ID, drop = TRUE)]

In [ ]:
pheno_cov <- fread("/mnt/project/pheno_cov/smoke_lung_cancer_copd.csv")

In [ ]:
df <- merge(pheno_cov, close_rel, by = "eid", all.x = TRUE)

In [ ]:
df[is.na(drop), drop := FALSE]

In [ ]:
df[drop == TRUE, .N]

[1] 66411

In [ ]:
df <- df[drop == FALSE]

# Format fields

In [ ]:
pc_names <- colnames(df)[grepl("^22009-0\\.", colnames(df))]
df_pc <- df[, ..pc_names]
colnames(df_pc) <- sprintf("pc%s", 1:ncol(df_pc))
head(df_pc)

pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc11,pc12,pc13,pc14,pc15,pc16,pc17,pc18,pc19,pc20
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-10.7447,4.69757,-2.427240,2.858830,-1.741290,0.119959,2.765020,-1.793400,-11.840700,2.528030,1.91106,2.046050,-0.130913,-1.3642200,3.482930,2.4514900,0.780783,-1.859190,1.238690,4.452950
-14.6587,3.07706,-0.386946,1.630550,-0.579281,1.339320,2.714810,0.103133,-0.496704,1.203490,3.19874,0.614306,-0.198499,-3.4399100,1.552400,0.8895370,-2.215440,-2.637290,0.316530,1.376280
-11.1732,5.37414,-1.763310,0.999203,-4.466730,-0.915626,1.425040,-0.900987,-1.205530,3.360040,2.69228,-1.301670,-2.000320,-1.7245700,-1.262170,-3.5555900,0.243108,-1.125780,-6.971570,3.349120
-13.5205,6.94157,-3.615700,4.091570,3.631150,-2.791230,1.064050,-3.698680,0.612475,-2.331500,-2.43125,2.344910,1.279260,0.0081137,-0.856328,-0.1551750,-2.399290,-2.562500,-0.927864,-0.163535
-13.8896,4.77904,-4.004370,4.648340,4.329800,2.237950,0.803446,3.650780,-2.878140,0.260571,1.46552,1.958430,-0.192651,6.0817600,1.408990,1.3856200,1.093850,-2.422320,-4.442640,-0.185869
-13.2771,7.10413,-3.615120,3.301260,3.541180,-0.453903,0.726986,0.601553,5.994300,0.464687,-2.10885,3.290110,-1.089410,4.8256100,-0.872009,-0.0443844,3.210770,-0.812609,0.078049,2.890790


In [25]:
cancer_icd10_names <- colnames(df)[grepl("^40006", colnames(df))]
df_cancer_icd10 <- df[, ..cancer_icd10_names]
has_lung_cancer_mat <- df_cancer_icd10[, lapply(.SD, function(col){grepl("C34", col)})]
has_lung_cancer <- apply(has_lung_cancer_mat, any, MARGIN = 1)

In [26]:
copd_raw <- df[["131492-0.0"]]
copd <- ifelse(
    copd_raw == "",
    FALSE,
    ifelse(
        copd_raw %in% c(
            "1900-01-01", # Code has no event date
            "1901-01-01", # Code has event date before participant's date of birth
            "1902-02-02", # Code has event date matching participant's date of birth
            "1903-03-03", # Code has event date after participant's date of birth and falls in the same calendar year as date of birth
            "1909-09-09", # Code has event date in the future and is presumed to be a place-holder or other system default
            "2037-07-07"  # Code has event date in the future and is presumed to be a place-holder or other system default
        ),
        NA,
        TRUE
    )
)

In [27]:
df_others <- df[
    , .(
        eid,
        bmi = `21001-0.0`,
        age = `21022-0.0`,
        lung_cancer = has_lung_cancer,
        previous_or_current_smoker = (`20116-0.0` != "Never"),
        smoking_pack_years = `20161-0.0`,
        copd = copd
    )
]

In [28]:
ret <- cbind(df_others, df_pc)

In [29]:
fwrite(ret, "smoke_lung_cancer_copd_analysis_ready.csv")

In [30]:
system("dx upload smoke_lung_cancer_copd_analysis_ready.csv --path pheno_cov/")

# For plink2 sample filtering

In [31]:
ret2 <- ret[, .(`#FID` = eid, ID = eid)]

In [32]:
fwrite(ret2, "smoke_lung_cancer_copd_sample_list.txt", sep = "\t")

In [33]:
system("dx upload smoke_lung_cancer_copd_sample_list.txt --path pheno_cov/")